In [7]:
import sys
import os
import random
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if par_dir not in sys.path:
    sys.path.append(par_dir)

In [8]:
# Dictionary mapping the ordinal labels to particle types 
LABEL_DICT = {0:"gamma", 1:"e", 2:"mu"}

# Fix the colour scheme for each particle type
COLOR_DICT = {"gamma":"red", "e":"blue", "mu":"green"}

npz_path = os.path.join(os.getcwd(), 'Index_Storage')

### Load ResNet output - Update the test dump location if needed

In [9]:
fprs = []
tprs = []
thresholds = []

run_id = "/20200511_151728"

dump_dir = "/home/cmacdonald/CNN/dumps"
dump_file = "/test_validation_iteration_dump.npz"

softmax_index_dict = {value:key for key, value in LABEL_DICT.items()}
    
test_dump_path = dump_dir + run_id + dump_file
test_dump_np = np.load(test_dump_path, allow_pickle=True)

res_predictedlabels = np.concatenate(list([batch_array for batch_array in test_dump_np['predicted_labels']]))
res_softmaxes  = np.concatenate(list([batch_array for batch_array in test_dump_np['softmax']]))
res_labels   = np.concatenate(list([batch_array for batch_array in test_dump_np['labels']]))
res_energies = np.concatenate(list([batch_array for batch_array in test_dump_np['energies']]))
res_rootfiles = np.concatenate(list([batch_array for batch_array in test_dump_np['rootfiles']]))
res_eventids = np.concatenate(list([batch_array for batch_array in test_dump_np['eventids']]))
#res_positions = test_dump_np['positions'].reshape(-1)
res_angles = np.concatenate(list([batch_array for batch_array in test_dump_np['angles']]))

### Load original test dataset (load full h5 and apply test indices)

In [10]:
# Import test events from h5 file
filtered_index = "/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz"
filtered_indices = np.load(filtered_index, allow_pickle=True)
test_filtered_indices = filtered_indices['test_idxs']

original_data_path = "/data/WatChMaL/data/IWCDmPMT_4pi_fulltank_9M.h5"
f = h5py.File(original_data_path, "r")

hdf5_event_data = (f["event_data"])
original_eventdata = np.memmap(original_data_path, mode="r", shape=hdf5_event_data.shape,
                                    offset=hdf5_event_data.id.get_offset(), dtype=hdf5_event_data.dtype)

original_eventids = np.array(f['event_ids'])
original_rootfiles = np.array(f['root_files'])
original_energies = np.array(f['energies'])
original_positions = np.array(f['positions'])
original_angles = np.array(f['angles'])
original_labels = np.array(f['labels'])
#filtered_eventdata = original_eventdata[test_filtered_indices]
filtered_eventids = original_eventids[test_filtered_indices]
filtered_rootfiles = original_rootfiles[test_filtered_indices]
filtered_energies = original_energies[test_filtered_indices]
filtered_positions = original_positions[test_filtered_indices]
filtered_angles = original_angles[test_filtered_indices]
filtered_labels = original_labels[test_filtered_indices]

Check that resnet data is in the same order

In [12]:
for i in range(len(res_eventids)): 
    assert res_eventids[i]==filtered_eventids[i]
    assert res_rootfiles[i]==filtered_rootfiles[i]
assert len(res_eventids) == len(filtered_eventids)
print("Success! Resnet output in same order as h5 test set")

Success! Resnet output in same order as h5 test set


Filter out the events that FiTQun failed on from the h5 data

In [13]:
fq_failed_idxs = np.load(os.path.join(npz_path,'fq_failed_idxs.npz'), allow_pickle = True)['failed_indices_pointing_to_h5_test_set']

In [14]:
sfiltered_eventids = np.delete(filtered_eventids, fq_failed_idxs)
sfiltered_rootfiles = np.delete(filtered_rootfiles , fq_failed_idxs)
sfiltered_energies = np.delete(filtered_energies, fq_failed_idxs)
sfiltered_positions = np.delete(filtered_positions, fq_failed_idxs)
sfiltered_angles = np.delete(filtered_angles, fq_failed_idxs,0)
sfiltered_labels = np.delete(filtered_labels, fq_failed_idxs)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


### Load the fiTQun output

In [15]:
# File paths for fiTQun results
fiTQun_e_path = "/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_fiTQun_e-.npz"
fiTQun_mu_path = "/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_fiTQun_mu-.npz"
fiTQun_gamma_path = "/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_fiTQun_gamma.npz"

# Load fiTQun results
f_e = np.load(fiTQun_e_path, allow_pickle=True)
f_mu = np.load(fiTQun_mu_path, allow_pickle=True)
f_gamma = np.load(fiTQun_gamma_path, allow_pickle=True)

### Now let's construct the FiTQun data in the same order as the h5 test set and ResNet output

In [16]:
fq_filename_original = (f_gamma['filename'],f_e['filename'],f_mu['filename'])
fq_eventids_original = ( f_gamma['eventid'],f_e['eventid'], f_mu['eventid'])
fq_flag_original = (f_gamma['flag'] ,f_e['flag'],f_mu['flag'])
fq_nll_original = (f_gamma['nLL'],f_e['nLL'],f_mu['nLL'])

In [19]:
fq_rootfiles = np.empty(len(sfiltered_rootfiles),dtype=object)
fq_eventids = np.zeros(len(sfiltered_rootfiles))
fq_flag = np.empty((len(sfiltered_rootfiles),2))
fq_nll = np.empty((len(sfiltered_rootfiles),2))

fq_mapping_indices = np.load(os.path.join(npz_path,'fq_mapping_idxs.npz'),allow_pickle=True)['arr_0']

pbar = ProgressBar(widgets=['Arranging FiTQun Data. Progress: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(sfiltered_rootfiles))
pbar.start()
for i,ptype in enumerate(sfiltered_labels):
    fq_rootfiles[i] = str(fq_filename_original[ptype][fq_mapping_indices[i]])
    fq_eventids[i] = fq_eventids_original[ptype][fq_mapping_indices[i]]
    fq_flag[i] = fq_flag_original[ptype][fq_mapping_indices[i]]
    fq_nll[i] = fq_nll_original[ptype][fq_mapping_indices[i]]
    pbar.update(i)
pbar.finish()

Arranging FiTQun Data. Progress: 100% [0000000000000000000000000] Time: 0:00:18


### Now let's again verify that the events are in the right order

In [20]:
pbar = ProgressBar(widgets=['Verification Progress: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(sfiltered_rootfiles))
pbar.start()
for i in range(len(sfiltered_rootfiles)):
    assert re.sub('_fiTQun','',fq_rootfiles[i].split('/')[-1]) == sfiltered_rootfiles[i].split('/')[-1], print(fq_rootfiles[i])
    assert fq_eventids[i] -1 == sfiltered_eventids[i]
    pbar.update(i)
pbar.finish()
print("Success! We now have a FiTQun output set in the same order as the h5 test set")

Success! We now have a FiTQun output set in the same order as the h5 test set


### Find the indices of flagged events

In [21]:
flagged_idxs = np.where((fq_flag[:,0] == 1.) | (fq_flag[:,1] == 1.))[0]

In [22]:
print("The fraction of events that were flagged is " + str(len(flagged_idxs)/len(fq_flag)))

The fraction of events that were flagged is 0.2193331384406174


### Filter out first FiTQun failed files, and then FiTQun flagged files, from ResNet output

In [23]:
fq_failed_idxs = np.load(os.path.join(npz_path,'fq_failed_idxs.npz'),allow_pickle=True)['failed_indices_pointing_to_h5_test_set']

res_filtered_predictedlabels =  np.delete(res_predictedlabels, fq_failed_idxs)
res_filtered_softmaxes  = np.delete(res_softmaxes,fq_failed_idxs,0)
res_filtered_labels   = np.delete(res_labels,fq_failed_idxs)
res_filtered_energies =  np.delete(res_energies,fq_failed_idxs)
res_filtered_rootfiles =  np.delete(res_rootfiles,fq_failed_idxs)
res_filtered_eventids =  np.delete(res_eventids,fq_failed_idxs)
res_filtered_angles =  np.delete(res_angles,fq_failed_idxs,0)

res_filtered_predictedlabels =  np.delete(res_filtered_predictedlabels,flagged_idxs)
res_filtered_softmaxes  = np.delete(res_filtered_softmaxes,flagged_idxs,0)
res_filtered_labels   = np.delete(res_filtered_labels,flagged_idxs)
res_filtered_energies =  np.delete(res_filtered_energies,flagged_idxs)
res_filtered_rootfiles =  np.delete(res_filtered_rootfiles,flagged_idxs)
res_filtered_eventids =  np.delete(res_filtered_eventids,flagged_idxs)
res_filtered_angles =  np.delete(res_filtered_angles,flagged_idxs,0)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  import sys
/opt/conda/lib/python3.6/site-p

### Do a sanity check comparing unflagged FQ events with our filtered ResNet output size

In [24]:
(fq_filename_original[0].shape[0] + fq_filename_original[1].shape[0] + fq_filename_original[2].shape[0] ) - len(flagged_idxs)

2614337

In [25]:
for data in (res_filtered_softmaxes, res_filtered_predictedlabels, res_filtered_labels, res_filtered_energies, res_filtered_rootfiles, res_filtered_eventids, res_filtered_angles):
    assert data.shape[0] == 2614337
print("Success! Resnet output size matches size of unflagged, successful FiTQun output")

Success! Resnet output size matches size of unflagged, successful FiTQun output


In [27]:
np.savez(os.path.join(os.getcwd(),'resnet_filtered_output.npz'), res_filtered_softmaxes=res_filtered_softmaxes,
                                                                 res_filtered_rootfiles=res_filtered_rootfiles,
                                                                 res_filtered_eventids=res_filtered_eventids,
                                                                 res_filtered_energies=res_filtered_energies,
                                                                 res_filtered_labels=res_filtered_labels,
                                                                 res_filtered_predictedlabels=res_filtered_predictedlabels,
                                                                 res_filtered_angles=res_filtered_angles
        )